### Problema y dataset

**Problema (necesidad real).** Los programas de promoción y prevención (EPS/Secretarías de Salud) necesitan **focalizar recursos** para reducir eventos cardiovasculares (infarto, ACV). Se requiere un **modelo de riesgo** que, a partir de indicadores personales simples, permita **priorizar adultos** con mayor probabilidad de enfermedad cardíaca para citarlos a tamizajes (ECG, perfil lipídico) y educación en hábitos.

**Dataset.** *Personal Key Indicators of Heart Disease (2022 UPDATE)*  
**Enlace:** https://www.kaggle.com/datasets/kamilpytlak/personal-key-indicators-of-heart-disease  
**Autor en Kaggle:** Kamil Pytlak  
**Fuente original:** CDC BRFSS (Behavioral Risk Factor Surveillance System)  
**Licencia:** CC0 (Public Domain)

**Por qué lo elegimos?.** Contiene indicadores personales **accionables** (edad, IMC, tabaquismo, actividad física, sueño, diabetes, etc.) adecuados para **modelar riesgo** y **diseñar intervenciones** de prevención. Tiene tamaño grande (~320k registros), esquema compacto (≈18 variables) y una etiqueta binaria clara (**HeartDisease**) que facilita el entrenamiento y la evaluación.

---

### Variables clave y utilidad en el caso de uso

| Variable | Tipo | Utilidad |
|---|---|---|
| **HeartDisease** (objetivo) | Binaria | Etiqueta de historia de enfermedad cardíaca para entrenar/validar el clasificador. |
| **AgeCategory** | Categórica | La edad es un determinante mayor del riesgo cardiovascular. |
| **Sex** | Categórica | Diferencias de riesgo y presentación clínica por sexo. |
| **BMI** | Numérica | IMC elevado aumenta el riesgo cardiometabólico. |
| **Smoking** | Binaria | Factor modificable clave; guía intervención conductual. |
| **AlcoholDrinking** | Binaria | Asociado a HTA y eventos cardiovasculares. |
| **Diabetic** | Categórica | Comorbilidad que multiplica el riesgo; prioriza seguimiento. |
| **PhysicalActivity** | Binaria | Factor protector; apoya segmentación de educación en hábitos. |
| **GenHealth** | Ordinal | Autopercepción de salud; proxy de multimorbilidad. |
| **SleepTime** | Numérica | Sueño insuficiente se asocia con mayor riesgo cardiometabólico. |
| **Stroke** | Binaria | Antecedente vascular mayor; alerta de muy alto riesgo. |
| **KidneyDisease** | Binaria | ERC comparte rutas con ECV; eleva riesgo. |
| **PhysicalHealth / MentalHealth** | Numéricas | Carga de síntomas; útil para estratificación. |
| **DiffWalking** | Binaria | Limitación funcional relacionada con eventos. |
| **Asthma / SkinCancer / Race** | Varias | Contexto clínico y demográfico para ajuste y auditoría de equidad. |

---

### Ficha para registro (Drive del curso)

- **Nombre del dataset:** Personal Key Indicators of Heart Disease (2022 UPDATE)  
- **Autor:** Kamil Pytlak  
- **Enlace:** https://www.kaggle.com/datasets/kamilpytlak/personal-key-indicators-of-heart-disease  
- **Grupo 6:** *Didier Johan Álvarez Bustamante y Jonathan Álvarez Bustamante*
- **Nota importante:** El anterior dataset fue adaptadado al modelo relacional en 4 datasets separados de la siguiente forma: condiciones_medicas, habitos, persona, salud_cardiaca. Con el fines meramente educativos y cumplir con los requerimientos de la rúbrica al diseñar la base de datos. Solo se uso el dataset del 2020 por ergonomía y fines prácticos y representaba menos variables.


## INSTALACIÓN DE LIBRERIAS

- `import sqlite3`: Módulo estándar para conectar y ejecutar SQL en **SQLite** (archivos `.db` o en memoria).
- `import pandas as pd`: Librería para **DataFrames**; leer/escribir datos (CSV/Excel/SQL) y transformarlos.
- `from sqlalchemy import create_engine`: Crea un **engine** de conexión a BD; permite que `pandas` use `read_sql` / `to_sql` de forma sencilla.

In [37]:
!pip install sqlalchemy


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python3 -m pip install --upgrade pip


In [29]:
import sqlite3
import pandas as pd
from sqlalchemy import create_engine



## CARGA DE ARCHIVOS CSV Y PREVISUALIZACIÓN DE DATOS.

- `pd.read_csv(..., delimiter=",")`: Carga **4 archivos CSV** (condiciones médicas, hábitos, datos de la persona y salud cardíaca) desde la ruta indicada y los guarda en **DataFrames**: `df1`, `df2`, `df3`, `df4`.

- `df*.head(5)`: Muestra las **primeras 5 filas** de cada DataFrame para **previsualizar** estructura, tipos y valores; no modifica los datos.


In [30]:
df1 = pd.read_csv("/workspaces/bigdata/src/bigdata/static/csv/heart2020_condiciones_medicas.csv", delimiter=",")
df2 = pd.read_csv("/workspaces/bigdata/src/bigdata/static/csv/heart2020_habitos.csv", delimiter=",")
df3 = pd.read_csv("/workspaces/bigdata/src/bigdata/static/csv/heart2020_persona.csv", delimiter=",")
df4 = pd.read_csv("/workspaces/bigdata/src/bigdata/static/csv/heart2020_salud_cardiaca.csv", delimiter=",")

In [31]:
df1.head(5)

,id_condicion,Stroke,Asthma,KidneyDisease,SkinCancer,id_persona
0,1,No,Yes,No,Yes,1
1,2,Yes,No,No,No,2
2,3,No,Yes,No,No,3
3,4,No,No,No,Yes,4
4,5,No,No,No,No,5


In [32]:
df2.head(5)

,id_habito,Smoking,AlcoholDrinking,PhysicalActivity,Diabetic,id_persona
0,1,Yes,No,Yes,Yes,1
1,2,No,No,Yes,No,2
2,3,Yes,No,Yes,Yes,3
3,4,No,No,No,No,4
4,5,No,No,Yes,No,5


In [33]:
df3.head(5)

,id_persona,Sex,AgeCategory,Race,BMI,SleepTime,GenHealth,PhysicalHealth,MentalHealth,DiffWalking
0,1,Female,55-59,White,16.60,5.0,Very good,3.0,30.0,No
1,2,Female,80 or older,White,20.34,7.0,Very good,0.0,0.0,No
2,3,Male,65-69,White,26.58,8.0,Fair,20.0,30.0,No
3,4,Female,75-79,White,24.21,6.0,Good,0.0,0.0,No
4,5,Female,40-44,White,23.71,8.0,Very good,28.0,0.0,Yes


In [34]:
df4.head(5)

,id_saludcardiaca,HeartDisease,id_persona
0,1,No,1
1,2,No,2
2,3,No,3
3,4,No,4
4,5,No,5


### CREACIÓN DE LA BASE DE DATOS.

- `con = sqlite3.Connection("pacientes_cardiacos.db")`: **Abre (o crea)** una base de datos SQLite llamada `pacientes_cardiacos.db` y devuelve una **conexión**.
- Con `con.cursor()` ejecutas SQL; con `con.commit()` guardas cambios; al final `con.close()`.


In [ ]:
con = sqlite3.Connection("pacientes_cardiacos.db")

### LLENADO DE TABLAS DE LA BASE DE DATOS CREADA.

- Escribe **cuatro DataFrames** (`df1`–`df4`) como **tablas** en `pacientes_cardiacos.db`.
- Usa `if_exists="replace"` → si la tabla ya existe, **la reemplaza**.
- Usa `index=False` → **no** guarda la columna índice de pandas en la tabla.
- Nombres de tablas creadas: `heart2020_condiciones_medicas`, `heart2020_habitos`, `heart2020_persona`, `heart2020_salud_cardiaca`.


In [ ]:

df1.to_sql("heart2020_condiciones_medicas.db", con=con, if_exists="replace", index=False)
df2.to_sql("heart2020_habitos.db",           con=con, if_exists="replace", index=False)
df3.to_sql("heart2020_persona.db",           con=con, if_exists="replace", index=False)
df4.to_sql("heart2020_salud_cardiaca.db",    con=con, if_exists="replace", index=False)

319795